# Lab 2: Language models

Laboration 2 in EDAN20 @ LTH - http://cs.lth.se/edan20/coursework/assignment-2/

Author: Jonatan Kronander

## Objectives
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment
* Optionally read a short article on the importance of corpora


## Collecting a corpus


#### 1. Collect a corpus of at least 750,000 words. Alternatively, you can retrieve a corpus of novels by Selma Lagerlöf from this URL: http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/Selma.txt .

In [1]:
from urllib.request import urlopen

text = urlopen("http://fileadmin.cs.lth.se/cs/Education/EDAN20/corpus/Selma.txt").read() # Open file and read
strText = str(text,'utf-8') # From bytes to string

In [2]:
text_file = open("Selma.txt", "w")
text_file.write(strText)
text_file.close()

You will check the number of words using the Unix command wc -w. 

In [3]:
import subprocess
cmd_0 = 'wc -w Selma.txt'
print(cmd_0)

p = subprocess.Popen(cmd_0, stdout=subprocess.PIPE, shell=True)
out_0, err = p.communicate() 
print("Number of words in text:" + str(out_0,'utf-8'))

wc -w Selma.txt
Number of words in text:  965943 Selma.txt



#### 2. Run the concordance program (https://github.com/pnugues/ilppp/tree/master/programs/ch02/python) to print the lines containing a specific word, for instance Nils.

In [4]:
pattern = "Nils"
width = " 10"
cmd_1 = "python concord.py Selma.txt "+pattern+width
print(cmd_1)

p = subprocess.Popen(cmd_1, stdout=subprocess.PIPE, shell=True)
out_1, err = p.communicate() 
print("\n ----Lines containing " + pattern + " is listed below---- \n")
print(str(out_1,'utf-8'))

python concord.py Selma.txt Nils 10

 ----Lines containing Nils is listed below---- 

Nils Holgersso
os Holger Nilssons LV. A
tt! Se på Nils gåsapåg! 
tt! Se på Nils Holgersso
ll Holger Nilsson i Väst
Jag heter Nils Holgersso
det värt, Nils Holgersso
att du är Nils gåsapåg, 
r han var Nils gåsapåg, 
r och hur Nils gåsapåg h
å han var Nils gåsapåg? 
tiden, då Nils Holgersso
honom vad Nils Holgersso
 året, då Nils Holgersso
osta dem. Nils Holgersso
 sägas om Nils Holgersso
 där stod Nils Holgersso
de syn på Nils Holgersso
ässen och Nils Holgersso
. Men vad Nils Holgersso
 du hört, Nils gåsapåg, 
och om då Nils Holgersso
och om nu Nils Holgersso
g ut, att Nils Holgersso
nte ville Nils Holgersso
iden, när Nils Holgersso
honom. Om Nils Holgersso
gen hand, Nils Holgersso
rumle ner Nils Holgersso
o, jag är Nils Holgersso
tiden, då Nils Holgersso
Det står: Nils Holgersso
fär innan Nils Holgersso
föda, men Nils Holgersso
 året, då Nils Holgersso
ssen över Nils Holgersso
 dag fick Nils Holgersso

#### 3. Run the tokenization program (https://github.com/pnugues/ilppp/tree/master/programs/ch05/python) on your corpus and count the words using the Unix sort and uniq commands.

In [5]:
def toLowercase(matchobj):
    """
    Helper function
    """
    return matchobj.group(1).lower()

In [6]:
import regex as re

cmd_2 = "python tokenizer.py < Selma.txt"
print(cmd_2)

p = subprocess.Popen(cmd_2, stdout=subprocess.PIPE, shell=True)
out_2, err = p.communicate() 

stringOut = str(out_2,'utf-8') # Bytes --> String

cleanString = re.sub(r'[^A-Za-z ]', '', stringOut) # Remove all non characters (-space)
cleanString = re.sub(r'(\p{Lu})', toLowercase, cleanString) # Lowercase all characters
nlString = re.sub(r'[ ]', r'\n', cleanString) # New line instead of space

text_file = open("selmaCorpus.txt", "w")
text_file.write(nlString)
text_file.close()

selmaString = open('selmaCorpus.txt', 'r')

selmaStringData = selmaString.readlines()
selmaStringData = [ x.rstrip() for x in selmaStringData ]

python tokenizer.py < Selma.txt


In [7]:
# Count words with unix
cmd_3 = "sort selmaCorpus.txt | uniq"
print(cmd_3)

p = subprocess.Popen(cmd_3, stdout=subprocess.PIPE, shell=True)
out_3, err = p.communicate() 

stringOut = str(out_3,'utf-8')

text_file = open("cleanSelmaCorpus.txt", "w")
text_file.write(stringOut)
text_file.close()

selmaString = open('cleanSelmaCorpus.txt', 'r')
data = selmaString.readlines()

corpusList = [ x.rstrip() for x in data ]

sort selmaCorpus.txt | uniq


In [8]:
corpusList[:10]

['',
 'a',
 'abborrar',
 'abborrarna',
 'abborre',
 'abborren',
 'abborrens',
 'abborrn',
 'abcbok',
 'abcdarierna']

In [9]:
print("There are " + str(len(corpusList)) + " words in the corpus.")

There are 38663 words in the corpus.


## Normalizing a corpus


#### 1. Write a program to insert \< s> and \< /s> tags to delimit sentences. You can start from the tokenization and modify it. Use a simple heuristics such as: a sentence starts with a capital letter and ends with a period. Estimate roughly the accuracy of your program.


In [10]:
import regex as re

cmd_4 = "python tokenizer.py < Selma.txt"
print(cmd_4)

p = subprocess.Popen(cmd_4, stdout=subprocess.PIPE, shell=True)
out_4, err = p.communicate() 

stringOut = str(out_4,'utf-8') # Bytes --> String
cleanString = re.sub(r'[^A-ZÅÄÖa-zåäö \.?!]', '', stringOut) # Remove all non characters (-space)
nlString = re.sub(r'[ ]', r'\n', cleanString) # New line instead of space

text_file = open("normSelmaCorpus.txt", "w")
text_file.write(nlString)
text_file.close()

normselmaString = open('normSelmaCorpus.txt', 'r')

normData = normselmaString.readlines()
normData = [ x.rstrip() for x in normData ]

python tokenizer.py < Selma.txt


In [11]:
normData[:20]

['Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Selma',
 'Lagerlöf',
 'Innehåll',
 'Den',
 'kristna',
 'dagvisan',
 '',
 'Sveriges',
 'karta',
 'I',
 '.',
 'Pojken',
 '',
 'Tomten']

#### 2. Modify your program to remove the punctuation signs and set all the text in lower case letters.

In [12]:
delimitNormData = normData.copy()
added = 0

for i, word in enumerate(normData):
    upperCase = re.findall(r'^[A-Z]', word) # Remove all non characters (-space)

    if upperCase:
        delimitNormData[i + added] = re.sub(r'(\p{Lu})', toLowercase, word)
        delimitNormData.insert(i + added, "<s>")
        added = added + 1
  
    dot = re.findall(r'\.', word) # Remove all non characters (-space)
    if dot:
        delimitNormData.insert(i + added + 1, "<\s>")
        del delimitNormData[i + added]
    """
    For removing characters other then letters
    noLetters = re.findall(r'\p{L}', delimitNormData[i + added])
    if not noLetters:
        del delimitNormData[i + added]
        added = added - 1
    """

#### 3. The result should be a normalized text without punctuation signs where all the sentences are delimited with < s> and < /s> tags.

In [13]:
sentences = []
flag = 0 # 0 = sentence in ongoing, 1 = finished sentence
sentence = ""

for word in delimitNormData:
    if word == "<s>":
        continue
    if word == "<\s>":
        flag = 1
        sentences.append("<s>" + sentence + "<\s>")
        sentence = ""
        continue
    sentence = sentence + " " + word

#### 4. The five last lines of the text should look like this:

< s > hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse < /s >

< s > när prästen sa detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg < /s >

< s > prästens ord tycktes redan ha gått i uppfyllelse < /s >

< s > där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad < /s >

< s > hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare < /s >

In [14]:
sentences[len(sentences)-5:]

['<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste  och sådan kärlek måste vändas i välsignelse<\\s>',
 '<s> när prästen sa detta  kom alla människor att se bort mot klara gulla  och de förundrade sig över vad de såg<\\s>',
 '<s> prästens ord tycktes redan ha gått i uppfyllelse<\\s>',
 '<s> där stod klara fina gulleborg ifrån skrolycka  hon  som var uppkallad efter själva solen  vid sina föräldrars grav och lyste som en förklarad<\\s>',
 '<s> hon var likaså vacker som den söndagen  då hon gick till kyrkan i den röda klänningen  om inte vackrare<\\s>']

 #### Estimate roughly the accuracy of your program.

In [15]:
dot = re.findall(r'[\.\!\?]', strText)

In [16]:
print("There are " + str(len(sentences)) + " found sentences and estimated sentences is " + str(len(dot)) + ". \nThis gives an estiamted accuaracy of " + str(len(sentences)/len(dot))) 

There are 56097 found sentences and estimated sentences is 63698. 
This gives an estiamted accuaracy of 0.88067129266225


## Counting unigrams and bigrams

#### 1. Read and try programs to compute the frequency of unigrams and bigrams of the training set: [Program folder] (https://github.com/pnugues/ilppp/tree/master/programs/ch05/python)

In [17]:
cmd_5 = "python count.py < Selma.txt"
print(cmd_5)

p = subprocess.Popen(cmd_5, stdout=subprocess.PIPE, shell=True)
out_5, err = p.communicate() 

stringOut = str(out_5,'utf-8') # Bytes --> String
lines = len(re.findall(r'\n', stringOut))

print("There are " + str(lines) + " unigrams in training set.")
print("Most common words:\n" + stringOut[:105])

python count.py < Selma.txt
There are 39356 unigrams in training set.
Most common words:
och 	 37799
att 	 28914
han 	 22743
det 	 22087
i 	 17072
som 	 16790
hade 	 14955
på 	 14634
hon 	 14093


In [18]:
cmd_6 = "python count_bigrams.py < Selma.txt"
print(cmd_6)

p = subprocess.Popen(cmd_6, stdout=subprocess.PIPE, shell=True)
out_6, err = p.communicate() 

stringOut = str(out_6,'utf-8') # Bytes --> String
lines = len(re.findall(r'\n', stringOut))

print("There are " + str(lines) + " bigrams in training set.")
print("Some bigrams:\n" + stringOut[:214])

python count_bigrams.py < Selma.txt
There are 319877 bigrams in training set.
Some bigrams:
3 	 ('nils', 'holgerssons')
1 	 ('holgerssons', 'underbara')
1 	 ('underbara', 'resa')
4 	 ('resa', 'genom')
3 	 ('genom', 'sverige')
1 	 ('sverige', 'selma')
11 	 ('selma', 'lagerlöf')
2 	 ('lagerlöf', 'innehåll')


#### 2. What is the possible number of bigrams and their real number? 

As there are 965 943 words in the corpus the maximum possible number of bigrams is 965 943 * 2 - 2 = 1 931 884

Their real value for selma corpus is, as printed above, 319 877. 

#### Explain why such a difference. What would be the possible number of 4-grams.

This gives a difference of (1 931 884 - 319 877 = ) 1 612 007 "missing" bigrams. This is because of there is many bigrams that are occuring twice. For example "selma lagerlöf" is occuring 11 times, as seen printed above. 

In [19]:
len(stringOut)

7569203

', 'storkarl')
1 	 ('som', 'väntades')
1 	 ('präst', 'än')
1 	 ('vanliga', 'som')
1 	 ('predika', 'de')
1 	 ('bodde', 'ju')
1 	 ('askedalarna', 'så')
1 	 ('hända', 'mycket')
1 	 ('socknen', 'utan')
1 	 ('det', 'begravningsfolket')
1 	 ('begravningsfolket', 'körde')
1 	 ('steg', 'sedan')
1 	 ('sedan', 'ur')
1 	 ('kärrorna', 'det')
1 	 ('talbacken', 'men')
1 	 ('annars', 'såg')
1 	 ('märkvärdigt', 'de')
1 	 ('mer', 'förundrade')
1 	 ('förundrade', 'men')
1 	 ('som', 'åker')
1 	 ('ett', 'begravningsfölje')
1 	 ('begravningsfölje', 'bör')
1 	 ('bör', 'ju')
1 	 ('har', 'del')
1 	 ('sorgen', 'kistan')
1 	 ('kistan', 'blev')
1 	 ('blev', 'nerflyttad')
1 	 ('nerflyttad', 'från')
1 	 ('från', 'flakvagnen')
1 	 ('nersatt', 'på')
1 	 ('bockar', 'som')
1 	 ('var', 'uppställda')
1 	 ('uppställda', 'på')
1 	 ('förhand', 'utanför')
1 	 ('sockenstugan', 'där')
1 	 ('till', 'stå')
1 	 ('klockorna', 'började')
1 	 ('ett', 'herrans')
1 	 ('väder', 'hela')
1 	 ('tiden', 'regnet')
2 	 ('regnet', 'kom')
1 	

In [26]:
print(965943)

965943


#### 3. Propose a solution to cope with bigrams unseen in the corpus. This topic will be discussed during the lab session.

## Computing the likelihood of a sentence


1. Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [Program folder]. (https://github.com/pnugues/ilppp/tree/master/programs/ch05/python)


2. Write a program to compute the sentence probability using bigrams.

3. Select five sentences in your test set and run your programs on them.

4. Tabulate your results as in the examples below with the sentence "Det var en gång en katt som hette Nils":


Unigram model
=====================================================
wi C(wi) #words P(wi)
=====================================================
det 22086 1086836 0.02032137323386417
var 12852 1086836 0.011825151172762036
en 13921 1086836 0.012808740233117047
gång 1332 1086836 0.0012255758918548888
en 13921 1086836 0.012808740233117047
katt 15 1086836 1.3801530313681181e-05
som 16790 1086836 0.015448512931113802
hette 107 1086836 9.845091623759242e-05
nils 84 1086836 7.728856975661462e-05
</s> 62283 1086836 0.057306714168467
=====================================================
Prob. unigrams:   4.4922846219128876e-27
Geometric mean prob.: 0.0023187115559242404
Entropy rate:   8.752460922513437
Perplexity:   431.2739967353978


Bigram model
=====================================================
wi wi+1 Ci,i+1 C(i) P(wi+1|wi)
=====================================================
<s> det 5913 62283 0.09493762342854391
det var 4023 22086 0.1821515892420538
var en 753 12852 0.05859010270774977
en gång 695 13921 0.04992457438402414
gång en 23 1332 0.017267267267267267
en katt 5 13921 0.0003591695998850657
katt som 2 15 0.13333333333333333
som hette 50 16790 0.0029779630732578916
hette nils 0 107 0.0 *backoff: 7.728856975661462e-05
nils </s> 2 84 0.023809523809523808
=====================================================
Prob. bigrams: 2.292224542392586e-19
Geometric mean prob.: 0.013678098151101147
Entropy rate: 6.191988542790593
Perplexity: 73.10957919390972
    

## Reading

As an application of n-grams, execute the Jupyter notebook by Peter Norvig here (http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb). Just run all the cells and be sure that you understand the code. You will find the data here (http://norvig.com/ngrams/). In you report, you will describe one experiment with a long string of words your will create yourself.

# Complement

As a complement, you can read a paper by Church (https://researcher.watson.ibm.com/researcher/view.php?person=us-kwchurch) and Hanks, Word Association Norms, Mutual Information, and Lexicography, (https://www.aclweb.org/anthology/J90-1003) Computational Linguistics, 16(1):22-29, 1990, as well as another one on backoff by Brants et al. (2007) Large language models in machine translation (https://www.aclweb.org/anthology/D07-1090).
